In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from IPython.display import display, clear_output
import plotly.io as pio     
import math
import time
import nbformat
import anywidget


## Setup MediaPipe


In [ ]:
mp_hands = mp.solutions.hands
# Diminuir confidence para melhorar a detecção
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Função para garantir que a câmera está funcionando
def check_camera():
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        print("ERRO: Não foi possível acessar a câmera!")
        return False
    camera.release()
    return True

# Verificar câmera
if not check_camera():
    print("Utilize a opção abaixo para modo de demonstração sem câmera")

def create_graph():
    G = nx.random_geometric_graph(10, 0.5, dim=3)
    pos = nx.get_node_attributes(G, 'pos')
    for k, v in pos.items():
        G.nodes[k]['pos'] = np.array(v)
    return G

def plot_3d_graph(G, camera=None):
    import plotly.io as pio
    # Configurar plotly para melhor performance em notebook
    #pio.renderers.default = "notebook_connected"

    # Extrair posições dos nós
    pos = nx.get_node_attributes(G, 'pos')

    # Criar as coordenadas dos nós
    Xn = [pos[k][0] for k in G.nodes()]
    Yn = [pos[k][1] for k in G.nodes()]
    Zn = [pos[k][2] for k in G.nodes()]

    # Criar as coordenadas das arestas
    Xe, Ye, Ze = [], [], []
    for e in G.edges():
        Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
        Ye.extend([pos[e[0]][1], pos[e[1]][1], None])
        Ze.extend([pos[e[0]][2], pos[e[1]][2], None])

    # Criar o grafo 3D
    trace_nodes = go.Scatter3d(x=Xn, y=Yn, z=Zn,
                               mode='markers',
                               marker=dict(size=8, color='blue'),
                               hoverinfo='text',
                               text=[f'Node {i}' for i in G.nodes()])

    trace_edges = go.Scatter3d(x=Xe, y=Ye, z=Ze,
                               mode='lines',
                               line=dict(width=1, color='black'),
                               hoverinfo='none')

    layout = go.Layout(
        title='Grafo 3D Interativo',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            camera=camera if camera else dict(eye=dict(x=1.25, y=1.25, z=1.25))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig = go.Figure(data=[trace_edges, trace_nodes], layout=layout)
    return fig


## Funções para detecção de gestos


In [ ]:
def is_hand_open(hand_landmarks):
    # Compara a posição da ponta dos dedos com as juntas inferiores para verificar se estão estendidos
    # Polegar (posição diferente por causa da orientação)
    thumb_tip = hand_landmarks.landmark[4].y < hand_landmarks.landmark[3].y

    # Outros dedos - verificar se estão estendidos (ponta acima da junta inferior)
    index_finger = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    middle_finger = hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y
    ring_finger = hand_landmarks.landmark[16].y < hand_landmarks.landmark[14].y
    pinky = hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y

    # Mão é considerada aberta se a maioria dos dedos estiver estendida
    fingers_extended = sum([thumb_tip, index_finger, middle_finger, ring_finger, pinky])
    return fingers_extended >= 4

def is_hand_closed(hand_landmarks):
    # Mão fechada: maioria dos dedos dobrados
    thumb_folded = hand_landmarks.landmark[4].y > hand_landmarks.landmark[3].y
    index_folded = hand_landmarks.landmark[8].y > hand_landmarks.landmark[6].y
    middle_folded = hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y
    ring_folded = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    pinky_folded = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y

    fingers_folded = sum([thumb_folded, index_folded, middle_folded, ring_folded, pinky_folded])
    return fingers_folded >= 4

def measure_pinch_distance(hand_landmarks):
    # Calcular a distância entre as pontas do polegar e do indicador
    thumb_tip = np.array([hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, hand_landmarks.landmark[4].z])
    index_tip = np.array([hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y, hand_landmarks.landmark[8].z])
    return np.linalg.norm(thumb_tip - index_tip)




## Inicia câmera e manipulação


In [ ]:

# Inicialização da câmera e variáveis globais
cap = cv2.VideoCapture(0)
prev_x, prev_y = None, None # prev_x, prev_y não estão sendo usados neste código, podem ser removidos se não forem necessários
camera_eye = dict(x=1.25, y=1.25, z=1.25)
G = create_graph()

# Aumentar sensibilidade para melhorar resposta dos gestos
rotation_speed = 10.0      # Aumentado de 0.5 em 0.5
movement_speed = 10.0      # Aumentado de 0.5 em 0.5
zoom_sensitivity = 10.0   # Aumentado de 1.0

# Variáveis para rastreamento de gestos
prev_hand_pos = None
prev_pinch_distance = None

# Configuração para atualização do grafo
update_graph = False

# Configurações iniciais para o grafo
# MUITO IMPORTANTE: Usar go.FigureWidget e display() para renderização dinâmica no notebook


fig = go.FigureWidget(plot_3d_graph(G, dict(eye=camera_eye)))
display(fig) # Exibe o gráfico uma única vez como um widget interativo

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Não foi possível ler o frame da câmera. Verifique se a câmera está conectada e acessível.")
            break

        # Processamento do frame para detecção de mãos
        #frame_rgb = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB) #para camera traseira, pois a frontal tem a questão do espelhamento

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #



        results = hands.process(frame_rgb)

        h, w, _ = frame.shape

        # Inicialize as variáveis para evitar NameError caso nenhuma mão seja detectada
        dx, dy = 0.0, 0.0
        hand_open, hand_closed = False, False
        pinch_distance = 0.0
        
        # Variável para o texto de debug do gesto
        current_gesture_status = "Nenhuma mão detectada"

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0] # Assumindo apenas 1 mão

            # Extrair posição da mão (usar ponto central da mão)
            wrist = hand_landmarks.landmark[0]
            hand_pos = np.array([wrist.x, wrist.y])

            # Detectar o tipo de gesto
            hand_open = is_hand_open(hand_landmarks)
            hand_closed = is_hand_closed(hand_landmarks)
            pinch_distance = measure_pinch_distance(hand_landmarks)

            if prev_hand_pos is not None:
                # Calcular movimento da mão
                dx = hand_pos[0] - prev_hand_pos[0]
                dy = hand_pos[1] - prev_hand_pos[1]

                # 1. Controle com mão aberta: rotacionar o grafo
                if hand_open:
                    camera_eye['x'] += dy * rotation_speed
                    camera_eye['y'] += dx * rotation_speed
                    update_graph = True
                    current_gesture_status = "Mão aberta (rotação)"

                # 2. Controle com mão fechada: mover no eixo X
                elif hand_closed:
                    # Mover a visualização (originalmente era Z, ajustei para Z para movimento vertical)
                    camera_eye['x'] += dx * movement_speed
                    camera_eye['z'] += dy * movement_speed # Mantive Z para movimento vertical como no último ajuste
                    update_graph = True
                    current_gesture_status = "Mão fechada (movimento)"

                # 3. Controle pinça (polegar + indicador): zoom
                if prev_pinch_distance is not None:
                    zoom_delta = prev_pinch_distance - pinch_distance
                    # Adicionar um pequeno limiar para evitar zoom indesejado por ruído
                    if abs(zoom_delta) > 0.005: # Limiar ajustável
                        zoom_factor = 1 + (zoom_delta * zoom_sensitivity)
                        camera_eye['x'] *= zoom_factor
                        camera_eye['y'] *= zoom_factor
                        camera_eye['z'] *= zoom_factor
                        update_graph = True
                        if not hand_open and not hand_closed: # Para não sobrescrever se já houver outro gesto primário
                            current_gesture_status = "Pinça (zoom)"

            # Atualizar posições anteriores para o próximo frame
            prev_hand_pos = hand_pos
            prev_pinch_distance = pinch_distance

            # Desenhar landmarks da mão
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        else: # Nenhuma mão detectada
            prev_hand_pos = None
            prev_pinch_distance = None
            current_gesture_status = "Nenhuma mão detectada" # Redefinir status quando nenhuma mão é detectada
        
       # ***** TODAS AS LINHAS DE TEXTO AGORA USAM 'frame' DIRETAMENTE *****
        # Adicionar textos de debug e instruções ao 'frame' (que já tem o exoesqueleto)
        cv2.putText(frame, "Mão aberta: rotação | Mão fechada: movimento | Pinça: zoom", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Gesto: {current_gesture_status}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Debug: dx={dx:.4f}, dy={dy:.4f}, Pinch={pinch_distance:.4f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        cv2.putText(frame, f"Hand Open Detected: {hand_open}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.putText(frame, f"Hand Closed Detected: {hand_closed}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        # *******************************************************************

        # ***** AQUI: CRIA 'frame_display' APÓS TODAS AS OPERAÇÕES NO 'frame' *****
        frame_display = cv2.flip(frame, 1) # Inverte horizontalmente para exibição espelhada
        # *************************************************************************

        # Atualizar o grafo se necessário (somente se update_graph for True)
        if update_graph:
            # Obter as novas coordenadas para os nós e arestas com base na nova câmera
            # Em vez de recriar a figura, vamos obter os dados dos traços diretamente
            # Reutilizamos a lógica de plot_3d_graph, mas sem criar uma *nova* Figure object
            
            pos = nx.get_node_attributes(G, 'pos')

            Xn = [pos[k][0] for k in G.nodes()]
            Yn = [pos[k][1] for k in G.nodes()]
            Zn = [pos[k][2] for k in G.nodes()]

            Xe, Ye, Ze = [], [], []

            for e in G.edges():
                Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
                Ye.extend([pos[e[0]][1], pos[e[1]][1], None])
                Ze.extend([pos[e[0]][2], pos[e[1]][2], None])

            # Assumimos que fig.data[0] é trace_edges e fig.data[1] é trace_nodes
            # Atualize os dados dos traços existentes
            with fig.batch_update(): # Use batch_update para eficiência
                fig.data[0].x = Xe # trace_edges.x
                fig.data[0].y = Ye # trace_edges.y
                fig.data[0].z = Ze # trace_edges.z

                fig.data[1].x = Xn # trace_nodes.x
                fig.data[1].y = Yn # trace_nodes.y
                fig.data[1].z = Zn # trace_nodes.z
                
                # Atualize a câmera do layout diretamente
                fig.layout.scene.camera.eye = camera_eye # camera_eye já é um dict com x,y,z
                fig.layout.title.text = 'Grafo 3D Interativo' # Opcional: pode manter o título original

            update_graph = False # Resetar o flag após a atualização


        # Mostrar frame da câmera
        #cv2.imshow("Camera - Controle do grafo - Pressione 'q' para sair", frame)

        cv2.imshow("Camera - Controle do grafo - Pressione 'q' para sair", frame_display)



        # Checar por saída do loop
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

except Exception as e:
    # Captura qualquer exceção não tratada no loop
    print(f"Ocorreu um erro no loop principal: {e}")
finally:
    # Garante que a câmera seja liberada e as janelas do OpenCV sejam fechadas ao sair
    cap.release()
    cv2.destroyAllWindows()
    print("Câmera liberada e janelas fechadas.")

    
    

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'black', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
              'uid': '6bca08fa-ad08-4365-a3a2-2c5005c29252',
              'x': [0.5072896074942608, 0.7819505874598202, None,
                    0.5072896074942608, 0.18614376511739872, None,
                    0.5072896074942608, 0.5643126810964567, None,
                    0.5072896074942608, 0.6549046596010838, None,
                    0.5072896074942608, 0.5494257607460109, None,
                    0.5072896074942608, 0.7003417064515015, None,
                    0.5072896074942608, 0.46959968361587845, None,
                    0.7819505874598202, 0.5127527048261582, None,
                    0.7819505874598202, 0.5643126810964567, None,
                    0.7819505874598202, 0.6549046596010838, None,
                    0.7819505874598202, 0.5494257607460109, None,
                    0.7819505874598202, 0

Câmera liberada e janelas fechadas.
